In [1]:
import numpy as np
import pandas as pd
import os

<h1>Daten Einlesen</h1>

In [2]:
unpickled = pd.read_pickle("../WI3_BusinessIntelligence_Data/data_inside_20211010_160005_one_hour_example.pkl")

print(unpickled)

{'accelerometer_x': array([-7.0458984, -7.0458984, -7.0654297, ..., -7.104492 , -7.104492 ,
       -7.060547 ], dtype=float32), 'accelerometer_y': array([6.9628906, 6.9628906, 6.982422 , ..., 6.9433594, 6.9433594,
       6.8896484], dtype=float32), 'accelerometer_z': array([0.7324219 , 0.7324219 , 0.7763672 , ..., 0.7910156 , 0.7910156 ,
       0.80078125], dtype=float32), 'magnetometer_x': array([-52250000., -54125000., -54125000., ..., -53437500., -54187500.,
       -54187500.], dtype=float32), 'magnetometer_y': array([4437500., -312500., -312500., ..., 1875000.,  750000.,  750000.],
      dtype=float32), 'magnetometer_z': array([-7750000., -4250000., -4250000., ..., -5875000., -3875000.,
       -3875000.], dtype=float32), 'audio': array([0.00689697, 0.00643921, 0.00540161, ..., 0.01193237, 0.01010132,
       0.01739502], dtype=float32), 'timer': array([      0,       1,       2, ..., 3599905, 3599905, 3599906],
      dtype=uint32)}


In [17]:
print(unpickled['timer'].max())
print(unpickled['timer'].size)

3599906
7415238


In [4]:
print(unpickled['audio'].size)

7560000


<h1>Audio Step für Smoothing festlegen</h1>

In [5]:
# to 20Hz
# Calculation:   step  =  2100d/s  /  20Hz  =  105d/hz

step = int(2100 / 20) 

print('steps:', step)

steps: 105


<h1>In einem Dataframe zusammenfassen</h1>

In [6]:
# Ein Dataframe für Audio
df_audio = pd.DataFrame(unpickled['audio'], columns=['audio'])
df_audio = df_audio[:-195]
# timer in Mikrosekunden
df_audio['timer'] = np.array(np.arange(0,(unpickled['timer'].max()+1),(1000/2100)))
print(df_audio)
# timer auf ganzzahl runden
df_audio['timer'] = df_audio['timer'].round().astype(int)
print(df_audio)

            audio         timer
0        0.006897  0.000000e+00
1        0.006439  4.761905e-01
2        0.005402  9.523810e-01
3        0.004669  1.428571e+00
4        0.003265  1.904762e+00
...           ...           ...
7559800  0.006287  3.599905e+06
7559801  0.018188  3.599905e+06
7559802  0.013062  3.599906e+06
7559803  0.007843  3.599906e+06
7559804  0.006104  3.599907e+06

[7559805 rows x 2 columns]
            audio    timer
0        0.006897        0
1        0.006439        0
2        0.005402        1
3        0.004669        1
4        0.003265        2
...           ...      ...
7559800  0.006287  3599905
7559801  0.018188  3599905
7559802  0.013062  3599906
7559803  0.007843  3599906
7559804  0.006104  3599907

[7559805 rows x 2 columns]


In [7]:
dict_no_audio = {}
for k in unpickled.keys():
    if k != 'audio':
        dict_no_audio[k] = unpickled[k]
     
print(dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################
# das hier ist ist hoffentlich nur für das 1h example notwendig
dict_no_audio['timer'] = np.append(dict_no_audio['timer'], 3599906)
print('after append: ', dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################

df_no_audio = pd.DataFrame(dict_no_audio)
df_no_audio

7415239  /  7415239  /  7415238
after append:  7415239  /  7415239  /  7415239


,accelerometer_x,accelerometer_y,accelerometer_z,magnetometer_x,magnetometer_y,magnetometer_z,timer
0,-7.045898,6.962891,0.732422,-52250000.0,4437500.0,-7750000.0,0
1,-7.045898,6.962891,0.732422,-54125000.0,-312500.0,-4250000.0,1
2,-7.065430,6.982422,0.776367,-54125000.0,-312500.0,-4250000.0,2
3,-7.104492,6.962891,0.776367,-54125000.0,-312500.0,-4250000.0,2
4,-7.104492,6.962891,0.776367,-54125000.0,-312500.0,-4250000.0,2
...,...,...,...,...,...,...,...
7415234,-7.109375,6.948242,0.815430,-53437500.0,1875000.0,-5875000.0,3599905
7415235,-7.109375,6.948242,0.815430,-53437500.0,1875000.0,-5875000.0,3599905
7415236,-7.104492,6.943359,0.791016,-53437500.0,1875000.0,-5875000.0,3599905
7415237,-7.104492,6.943359,0.791016,-54187500.0,750000.0,-3875000.0,3599906


In [20]:
s_df = pd.merge(df_audio, df_no_audio, how='left', on='timer')
print(s_df)

             audio    timer  accelerometer_x  accelerometer_y  \
0         0.006897        0        -7.045898         6.962891   
1         0.006439        0        -7.045898         6.962891   
2         0.005402        1        -7.045898         6.962891   
3         0.004669        1        -7.045898         6.962891   
4         0.003265        2        -7.065430         6.982422   
...            ...      ...              ...              ...   
16225858  0.013062  3599906        -7.104492         6.943359   
16225859  0.013062  3599906        -7.060547         6.889648   
16225860  0.007843  3599906        -7.104492         6.943359   
16225861  0.007843  3599906        -7.060547         6.889648   
16225862  0.006104  3599907              NaN              NaN   

          accelerometer_z  magnetometer_x  magnetometer_y  magnetometer_z  
0                0.732422     -52250000.0       4437500.0      -7750000.0  
1                0.732422     -52250000.0       4437500.0      -775

<h1>Aufteilen in teil Arrays und berechnen der Werte</h1>

In [16]:
###################### nach MS ################################################
# erstellen eines leeren dictionaries
smt_dict = {}

for variable in s_df.keys():
    if "accelerometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ac_split_arr = np.split(s_df[variable], range(step, s_df['timer'].size, step))
        pad = len(max(ac_split_arr, key=len))
        pad_ac_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ac_split_arr])
        stack_ac_split_arr = np.vstack(pad_ac_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.nanmax(stack_ac_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.nanquantile(stack_ac_split_arr, 0.95, axis = 1)
        # minimum
        smt_dict[variable+'_min'] = np.nanmin(stack_ac_split_arr, axis = 1)
        # 5% quantile
        smt_dict[variable+'_05q'] = np.nanquantile(stack_ac_split_arr, 0.05, axis = 1)
        
    elif "magnetometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ma_split_arr = np.split(s_df[variable], range(step, s_df['timer'].size, step))
        pad = len(max(ma_split_arr, key=len))
        pad_ma_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ma_split_arr])
        stack_ma_split_arr = np.vstack(pad_ma_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # mean
        smt_dict[variable+'_mean'] = np.nanmean(stack_ma_split_arr, axis = 1)
        # median
        smt_dict[variable+'_med'] = np.nanmedian(stack_ma_split_arr, axis = 1)
        
    elif "audio" in variable:
        au_split_arr = np.split(s_df[variable], range(step, s_df['timer'].size, step))
        pad = len(max(au_split_arr, key=len))
        pad_au_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in au_split_arr])
        stack_au_split_arr = np.vstack(pad_au_split_arr)
        
        # Zusammenfassen und in DataFrame schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.nanmax(stack_au_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.nanquantile(stack_au_split_arr, 0.95, axis = 1)

        
print('sensor ', smt_dict['accelerometer_x_max'].size)
print('audio ', smt_dict['audio_max'].size)

/tmp/user/1019/ipykernel_22557/426909027.py:15: RuntimeWarning: All-NaN slice encountered
  smt_dict[variable+'_max'] = np.nanmax(stack_ac_split_arr, axis = 1)
/tmp/user/1019/ipykernel_22557/426909027.py:19: RuntimeWarning: All-NaN slice encountered
  smt_dict[variable+'_min'] = np.nanmin(stack_ac_split_arr, axis = 1)
/tmp/user/1019/ipykernel_22557/426909027.py:32: RuntimeWarning: Mean of empty slice
  smt_dict[variable+'_mean'] = np.nanmean(stack_ma_split_arr, axis = 1)


sensor  154533
audio  154533
